# Imports

In [199]:
import os
import pandas
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler

# Parsing data

In [200]:
data = pandas.read_csv("../data/dataNames.csv", header=0)
X = data.iloc[:, 2:] # La data sans id et diagnosis
Y = data['Diagnosis'].map({'B': 0, 'M': 1})

# numpy arrays :
X = X.to_numpy()
Y = Y.to_numpy().reshape(-1, 1)

# Normalisation des donnees
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


print(X.shape)
print(Y.shape)

(569, 30)
(569, 1)


# Creating perceptron

In [247]:
class MultilayerPerceptron :
	def __init__(self, X, Y):
		self.W = np.random.randn(X.shape[1], 1)
		self.b = np.random.randn(1)
		self.X = X
		self.Y = Y
	
	def model(self) :
		self.Z = self.X.dot(self.W) + self.b
		self.A = 1 / (1 + np.exp(-self.Z))

	def log_loss(self) :
		self.model()
		eps = 1e-15 
		A_clipped = np.clip(self.A, eps, 1 - eps) # pour éviter log(0) (empeche les valeures d'etre en dehors de l'interval 𝐴∈[10**−15,1−10**−15]
		result = -1 / self.X.shape[0] * np.sum(self.Y * np.log(A_clipped) + (1 - self.Y) * np.log(1 - A_clipped))
		return result

	def gradient_descent(self, learning_rate) :
		self.model()
		self.W -= learning_rate * (-1/self.X.shape[0] * np.dot(self.X.T, self.Y - self.A))
		self.b -= learning_rate * (-1/self.X.shape[0] * np.sum(self.Y - self.A))

# Revoir la vectorisation des differentes equations et noter dans la doc leur origine

# Testing perceptron

In [252]:
m = MultilayerPerceptron(X_scaled, Y)
m.model()
print(m.log_loss())
for _ in range(0, 10000) :
	m.gradient_descent(0.1)
	print(m.log_loss())


3.072442799244543
2.5089564010536742
2.035371764532777
1.6660904119222726
1.389029450337972
1.1833256212622345
1.0309990599571117
0.9175155120209949
0.831387235965864
0.7642156633563868
0.7102544939138251
0.6657253356468775
0.6281446905119473
0.5958371537580585
0.5676349638771807
0.5426990296789247
0.5204102811205908
0.5003015324218812
0.48201334290624276
0.46526455638689956
0.44983208920627366
0.43553670824497026
0.4222327930380266
0.40980081831178533
0.3981417416448163
0.387172756507581
0.3768240433491978
0.36703626163520325
0.3577585982612777
0.34894723700956554
0.3405641483652477
0.33257612414952253
0.32495400011678793
0.3176720237494349
0.31070733514820864
0.30403953697664315
0.29765033544282277
0.2915232387866646
0.28564330303162055
0.27999691716971514
0.27457162170184246
0.26935595572469323
0.26433932866754845
0.2595119134245076
0.25486455806818614
0.2503887136230155
0.24607637556775952
0.2419200368696676
0.2379126504654161
0.23404759922271107
0.23031867155939006
0.2267200410694

## LogLoss tres bas alors que probleme pas lineaire, a voir pourquoi en implementant la fonction accuracy